In [3]:
from src import utils
import os
import pandas as pd
import numpy as np

In [9]:
datafile_path = os.path.join(os.path.abspath(os.getcwd()), 
                        ('data/server/')) 

In [10]:
print(datafile_path)

/data_disk/home/joongho/FL/data/server/


In [11]:
train_DS, val_DS = utils.data_loader(datafile_path)

Found 50000 files belonging to 10 classes.
Using 40000 files for training.
Found 50000 files belonging to 10 classes.
Using 10000 files for validation.


In [12]:
print(train_DS)

<MapDataset element_spec=(TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>


In [13]:
print(val_DS)

<MapDataset element_spec=(TensorSpec(shape=(None, 100, 100, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>
